In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy import stats
import geopandas as gpd
from shapely.geometry import Point
import folium
from datetime import datetime, timedelta
import re

In [68]:
hub = pd.read_excel("data/Trash Hauler Report - Nov 2017 - Nov 2019 Final.xlsx", skiprows=1)

In [69]:
# #rename the columns
hub.columns = [y.lower() for y in [x.strip(' ') if x[-1:] == " " else re.sub(r'\W', '_', x) for x in hub.columns]]

In [70]:
metro = pd.read_csv("data/hubNashville__311__Service_Requests.csv", low_memory=False)

In [71]:
metro.columns = ['request_number' if "#" in x else re.sub(r'\W+', '_', x).lower() for x in metro.columns]

In [72]:
#This fuction works but it is time consuming 
# inProj = Proj(init='epsg:3857')
# outProj = Proj(init='epsg:4326')
# [transform(x, y) for x in hub.state_plan_x for y in hub.state_plan_y]

In [73]:
hub_geo = pd.merge(hub, metro[['request_number', 'latitude', 'longitude']], how='inner', on='request_number')
hub_geo = hub_geo.drop(['state_plan_x', 'state_plan_y'], axis=1)
hub_geo.loc[:, ('geometry')] = hub_geo.apply(lambda x: Point(x.longitude, x.latitude), axis=1)
hub_geo = gpd.GeoDataFrame(hub_geo, crs = {'init': 'epsg:4326'}, geometry = hub_geo['geometry'])

**Filter the misses**

In [74]:
hub_geo['description'] = hub_geo['description'].astype(str)
hub_geo['request'] = hub_geo['request'].astype(str)

In [75]:
hub_geo['request_description'] = (hub_geo.request + hub_geo.description).str.lower()

In [76]:
hub_miss_geo = hub_geo[hub_geo['request_description'].str.contains(r'miss', na=False)]

In [77]:
hub_geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 20226 entries, 0 to 20225
Data columns (total 13 columns):
request_number         20226 non-null int64
date_opened            20226 non-null datetime64[ns]
request                20226 non-null object
description            20226 non-null object
incident_address       20217 non-null object
zip_code               20151 non-null float64
trash_hauler           19325 non-null object
trash_route            19279 non-null object
council_district       20177 non-null float64
latitude               20201 non-null float64
longitude              20201 non-null float64
geometry               20226 non-null object
request_description    20226 non-null object
dtypes: datetime64[ns](1), float64(4), int64(1), object(7)
memory usage: 2.2+ MB


**standardize the address column**

In [78]:
hub_miss_geo.loc[:, ('incident_address')] = hub_miss_geo['incident_address'].astype(str).str.lower()

D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [79]:
# # split the address into two groups by the first',', the first group is the street type and the second is the content after',' 
# matches = [re.search(r'(\w+),\s(.+)', x) if "," in x else re.search(r'(\w+)$|\W+$()', x) for x in hub_miss_geo.incident_address]

# #assign the groups
# for col, group in {'address_a': 1, 'address_b':2}.items():
#     hub_miss_geo[col] = [x.group(group) if x != None else None for x in matches]

In [80]:
# strip remove the leading and trailing characters only
hub_miss_geo.loc[:,('incident_address')] = \
    hub_miss_geo['incident_address'].replace(to_replace= r",.+", value='', regex=True).str.strip()

In [81]:
suffix = pd.read_excel('data/street_suffix.xlsx')
suffix = suffix[['abbrv','street']].dropna(how='any',axis=0).reset_index(drop=True)
suffix.abbrv = suffix.abbrv.str.lower()
suffix.street = suffix.street.str.lower()
suffix.head(1)

,abbrv,street
0,aly,allee alley ally aly


In [82]:
def address_standard():
    """
    replace all street types with their abbreviations:
    1. search the street types which are the last few letters in each cell 
       in the address column;
    2. extract the matched groups and compare the suffix dataframe which
       contains the street abbrevations and the related types;
    3. replace all street types with their abbreviations.
    """
# ave s and avenue s
    address = []
    for street in hub_miss_geo.incident_address:
        st_suffix = re.search(r'(\w+)$|\W+$|nan', street).group(1)
        for key, value in suffix.street.items():
            value_list = value.split()
            if st_suffix in value_list:
                st_suffix = suffix.abbrv[key]
            break 
        address.append(re.sub(r'\w+$|\W+$|nan', ' {}'.format(st_suffix), street))
    return (address)

In [83]:
hub_miss_geo.loc[:, ('incident_address')] = address_standard()

In [84]:
hub_miss_geo.incident_address.value_counts()

110 george l davis  blvd     28
5135 hickory hollow  pkwy    23
12546 old hickory  blvd      21
6007 obrien  ave             20
3710 n natchez  ct           20
                             ..
1654 pearl  st                1
4416 bench mark  dr           1
4125 wallace  ln              1
3839 crouch  dr               1
350 beech  ave                1
Name: incident_address, Length: 11197, dtype: int64

**dedup misses**

In [85]:
hub_miss_geo.trash_route = [str(x) if type(x) != float else x for x in hub_miss_geo.trash_route]

D:\Anaconda3\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [86]:
hub_miss_geo['pickup_day'] = [int(x[1]) if type(x) != float else x for x in hub_miss_geo.trash_route]

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [87]:
hub_miss_geo['weekday'] = [x.weekday() for x in hub_miss_geo.date_opened]

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [88]:
hub_miss_geo["differ"] = (hub_miss_geo.weekday - hub_miss_geo.pickup_day + 1) % 7

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [89]:
def pickup_date():
    p_date = []
    for x in hub_miss_geo.date_opened:
      for y in hub_miss_geo.differ:
        if math.isnan(y):
            a = y
        else:
            a = x - timedelta(days = y)
        break
      p_date.append(a)
    return p_date

In [90]:
hub_miss_geo['pickup_date'] = pickup_date()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [91]:
dedup_miss = hub_miss_geo.groupby('pickup_date').first().reset_index()

In [94]:
dedup_miss.shape

(17059, 17)

**count misses over a rolling 6 months period**

In [158]:
dedup_miss.incident_address.dropna()

0         4028 clarksville  pike
1           6528 thunderbird  dr
2          2603 old matthews  rd
3                 604 croley  dr
4          1816 jo johnston  ave
                  ...           
17054    2731 murfreesboro  pike
17055             1621 long  ave
17056        2943 windemere  cir
17057    3325 murfreesboro  pike
17058           604 somerset  ct
Name: incident_address, Length: 17059, dtype: object

In [157]:
dedup_miss.incident_address.dtype

dtype('O')

In [159]:
dedup_miss.groupby('incident_address').rolling(on = 'date_opened', window = '180D').request_number.count()

incident_address  date_opened        
 107              2018-09-10 15:57:00    1.0
 nan              2017-12-13 08:47:00    1.0
                  2017-12-29 13:09:00    2.0
                  2018-01-17 15:10:00    3.0
                  2018-01-24 12:38:00    4.0
                                        ... 
tusculum  rd      2019-07-18 19:27:00    1.0
                  2019-08-01 21:02:00    2.0
                  2019-08-02 13:52:00    3.0
westboro  dr      2019-08-15 10:42:00    1.0
xavier  dr        2017-12-30 23:43:00    1.0
Name: request_number, Length: 17059, dtype: float64

**identify instances where contractors missed an entire street or route in a month by contractor**

In [164]:
# filter then dedup or dedup first
hub_miss_entire = dedup_miss[dedup_miss['description'].str.contains(r'whole\s\w+|entire\s\w+|other\s\w+', na=False)]

In [165]:
# 1968
hub_miss_entire.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1772 entries, 21 to 17055
Data columns (total 17 columns):
pickup_date            1772 non-null datetime64[ns]
request_number         1772 non-null int64
date_opened            1772 non-null datetime64[ns]
request                1772 non-null object
description            1772 non-null object
incident_address       1772 non-null object
zip_code               1768 non-null float64
trash_hauler           1697 non-null object
trash_route            1700 non-null object
council_district       1769 non-null float64
latitude               1770 non-null float64
longitude              1770 non-null float64
geometry               1770 non-null object
request_description    1772 non-null object
pickup_day             1700 non-null float64
weekday                1772 non-null int64
differ                 1700 non-null float64
dtypes: datetime64[ns](2), float64(6), int64(2), object(7)
memory usage: 249.2+ KB


In [166]:
hub_miss_entire['area'] = [re.search(r'\d*\s?(.+)', x).group(1) for x in hub_miss_entire.incident_address]

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [167]:
hub_miss_entire.area.value_counts()

old hickory  blvd    18
wild oaks  ct        15
heritage  dr         14
buena vista  pike    10
claymille  ct        10
                     ..
w marthona  rd        1
dew st  a             1
nunley  dr            1
riverpoint  ct        1
 leatherbury          1
Name: area, Length: 994, dtype: int64

In [168]:
hub_miss_entire['month_year'] = hub_miss_entire.date_opened.dt.to_period('M')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [169]:
#26391
hub_miss_entire.groupby(['trash_hauler', 'month_year', 'area']).first()

pickup_date  request_number  \
trash_hauler month_year area                                                 
METRO        2017-11    franklin  ave  2017-11-05 10:18:00           26233   
                        iverson  ave   2017-11-02 16:30:00           26038   
                        park  cir      2017-11-21 15:09:00           29677   
             2017-12    ashwood  ave   2017-12-20 10:12:00           34633   
                        blue ridge  dr 2017-12-28 12:17:00           35835   
...                                                    ...             ...   
WASTE IND    2019-10    32nd ave  n    2019-10-09 10:31:00          257577   
                        3rd ave  n     2019-10-14 20:47:00          259664   
                        5th ave  n     2019-10-03 16:23:00          255657   
                        6th ave  n     2019-10-17 12:14:00          261348   
                        martin  st     2019-10-13 10:59:00          258825   

                                               date_opened  \
trash_hauler month_year area                                 
METRO        2017-11    franklin  ave  2017-11-06 10:18:00   
                        iverson  ave   2017-11-03 16:30:00   
                        park  cir      2017-11-22 15:09:00   
             2017-12    ashwood  ave   2017-12-21 10:12:00   
                        blue ridge  dr 2017-12-29 12:17:00   
...                                                    ...   
WASTE IND    2019-10    32nd ave  n    2019-10-10 10:31:00   
                        3rd ave  n     2019-10-15 20:47:00   
                        5th ave  n     2019-10-04 16:23:00   
                        6th ave  n     2019-10-18 12:14:00   
                        martin  st     2019-10-14 10:59:00   

                                                                     request  \
trash_hauler month_year area                                                   
METRO        2017-11    franklin  ave   Trash - Curbside/Alley Missed Pickup   
                        iverson  ave    Trash - Curbside/Alley Missed Pickup   
                        park  cir       Trash - Curbside/Alley Missed Pickup   
             2017-12    ashwood  ave    Trash - Curbside/Alley Missed Pickup   
                        blue ridge  dr  Trash - Curbside/Alley Missed Pickup   
...                                                                      ...   
WASTE IND    2019-10    32nd ave  n     Trash - Curbside/Alley Missed Pickup   
                        3rd ave  n      Trash - Curbside/Alley Missed Pickup   
                        5th ave  n      Trash - Curbside/Alley Missed Pickup   
                        6th ave  n      Trash - Curbside/Alley Missed Pickup   
                        martin  st                Trash Collection Complaint   

                                                                              description  \
trash_hauler month_year area                                                                
METRO        2017-11    franklin  ave   customer says missed trash pickup for the enti...   
                        iverson  ave                             entire street was missed   
                        park  cir       customer says the trash is usually missed ever...   
             2017-12    ashwood  ave            missed trash pickup for the entire street   
                        blue ridge  dr  missed pick up- caller states the entire stree...   
...                                                                                   ...   
WASTE IND    2019-10    32nd ave  n     missed entire alley, possibly due to illegal i...   
                        3rd ave  n      Trash not picked up, recycling misses every ot...   
                        5th ave  n                                      miss entire alley   
                        6th ave  n                  entire alley missed for several weeks   
                        martin  st      Trailer park all missed tras

**identify property damage, oil or hydraulic leaks in one month by contractor**

In [35]:
hub_damage = hub_geo[hub_geo['request_description'].str.contains(r'damage|oil|hydraulic|leak', na=False)]

In [163]:
hub_damage.head()

,request_number,date_opened,request,description,incident_address,zip_code,trash_hauler,trash_route,council_district,latitude,longitude,geometry,request_description,month_year
6,25325,2017-11-01 09:29:00,Damage to Property,Trash/emptied Wednesday & now metal black-mail...,4721 Chalmers Dr,37215.0,RED RIVER,3303,34.0,36.08074,-86.81965,POINT (-86.81965 36.08074000000001),damage to propertytrash/emptied wednesday & no...,2017-11
173,26971,2017-11-08 14:21:00,Damage to Property,truck is cutting into yard and damaging lawn,717 garrison dr,37207.0,RED RIVER,3205,2.0,36.22211,-86.81766,POINT (-86.81766 36.22211),damage to propertytruck is cutting into yard a...,2017-11
218,27615,2017-11-13 10:42:00,Trash Collection Complaint,"Trash/cart damage, due to trash/truck that sla...",3807 Richland Ave,37205.0,METRO,9307,24.0,36.13335,-86.83204,POINT (-86.83204000000001 36.13335),"trash collection complainttrash/cart damage, d...",2017-11
257,28158,2017-11-15 08:55:00,Damage to Property,cable lines pulled from house - caused damage ...,6347 alamo pl,37209.0,RED RIVER,4203,20.0,36.15274,-86.87178,POINT (-86.87178 36.15274),damage to propertycable lines pulled from hous...,2017-11
268,28291,2017-11-15 15:35:00,Trash Collection Complaint,The trash trucks in their area have been leaki...,711 Starlit Rd,37205.0,RED RIVER,1305,23.0,36.12747,-86.88654,POINT (-86.88654 36.12747),trash collection complaintthe trash trucks in ...,2017-11


In [139]:
hub_damage['month_year'] = hub_damage.date_opened.dt.to_period('M')

In [ ]:
hub_damage.mon